# 0. 드라이브 연결

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 1. jsonl 형식 데이터들 load

In [28]:
from datasets import load_dataset

dataset = load_dataset(
    "json",
    data_files={"train": "/content/drive/MyDrive/DILAB/HJ/News_DILAB/news_data/*.jsonl"}   # 구글 드라이브 데이터 경로 지정
)["train"]
print(dataset)



Dataset({
    features: ['title', 'body'],
    num_rows: 36
})


# 2. 불러온 데이터들을 Train/Valid로 분할

In [29]:
# .train_test_split() : 하나의 Dataset을 train용과 test용으로 분리하는 함수
  # test_size = 0.1 : test 사이즈를 10, train 사이즈를 90으로 설정함
  # seed = 42 : shuffle할 시드값을 지정한 것이다. 해당 시드값을 지정해놓으면, 다시 shuffle해도 똑같은 결과값이 나온다.
splits = dataset.train_test_split(test_size=0.1, seed=42)  # 90/10
train_dataset = splits["train"]
valid_dataset = splits["test"]

# 3. train/valid 데이터들 토크나이징

In [30]:
from transformers import AutoTokenizer

# 저장해둔 모델로부터 토크나이저 가져오기
model_path = "/content/drive/MyDrive/DILAB/llama3-Korean-Bllossom-8B"
tokenizer = AutoTokenizer.from_pretrained(model_path)

# 스페셜 토큰 등록하기
specials = {"additional_special_tokens" : ["<title>", "</title>", "<body>", "</body>"]}
tokenizer.add_special_tokens(specials)

  # to_one_batch() : 제목/본문을 하나의 입력 문자열로 합치는 함수
def to_one_batch(ex):
  return {"text" : "<title>" + ex["title"] + "</title>" + "<body>" + ex["body"] + "</body>"}


# 불러온 데이터들에 to_one_batch 함수 적용하기
  # .map() 함수는 인자로 지정해놓은 함수를 거친 결과를 기존 데이터셋에 추가하는 함수이다.
  # 따라서, "remove_columns"를 지정하지 않으면 해당 데이터셋에 있는 "title"과 "body"가 그대로 남아있으므로, remove_columns를 지정하는 것이다.
  # 이렇게 되면 해당 데이터셋에는 "text" 컬럼의 데이터만 남게 된다.
train_dataset = train_dataset.map(to_one_batch, remove_columns=train_dataset.column_names)
valid_dataset = valid_dataset.map(to_one_batch, remove_columns=valid_dataset.column_names)

  # tokenize_function() : 인자로 들어오는 batch를 토크나이즈 하는 함수
max_length = 2048 # 생성할 토큰 최대 길이
def tokenize_function(batch):
  return tokenizer( # 사전학습된 토크나이저 객체
    batch["text"], # 변환할 원본 텍스트 (문자열 or 문자열 리스트)
    truncation=True, # 문장이 너무 길면 max_length에 맞춰 잘라낼지 여부 결정(절단)
    padding = "max_length", # 문장이 짧은 경우, max_length 길이에 맞춰서 뒤에 [PAD] 토큰 붙여서 길이 맞추기
    max_length=max_length, # 최대 토큰 길이 설정
    return_tensors = "pt" # tokenizer는 파이썬의 dict 형태로 결과를 주는데, 해당 return을 Pytorch Tensor 형태로 반환하도록 설정
  )


train_tokenized = train_dataset.map(tokenize_function, batched=True) # 한번에 여러 샘플 묶음(배치)을 리스트/딕셔너리 형태로 전달하기 위한 설정
valid_tokenized = valid_dataset.map(tokenize_function, batched=True)

#print(train_tokenized['input_ids'])



Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

# 4. 토크나이즈한 값들 드라이브에 저장

In [31]:
train_tokenized.save_to_disk("/content/drive/MyDrive/DILAB/HJ/News_DILAB/train_tokenized")
valid_tokenized.save_to_disk("/content/drive/MyDrive/DILAB/HJ/News_DILAB/valid_tokenized")

Saving the dataset (0/1 shards):   0%|          | 0/32 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/4 [00:00<?, ? examples/s]